# Menu
<a name="navegacao"></a>

## 1) [Organização dos dados de Loteria](#empresas)
- 1.1 [Importação dos dados](#loopempr)
- 1.2 [Tratamento de Data](#cpfmei)


## 2) [Importação dos dados de datas hebraicas](#filiais)
- 2.1 [Loop de concatenação](#loopfil)

## 3) [Tratamento de Guematria](#sócios)

- 3.1 [Loop de concatenação](#loopscs)

In [1]:
%%time
import os
import sys
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from hebrew import Hebrew
from hebrew import GematriaTypes
from hebrew_numbers import gematria_to_int
import unicodedata
from pyluach import dates, hebrewcal, parshios
from hebrew_numbers import gematria_to_int


CPU times: total: 4.75 s
Wall time: 6.42 s


In [2]:
#! pip install tensorflow

In [3]:
sena = pd.read_excel("Mega-Sena.xlsx")

In [4]:
sena.dtypes

Concurso             int64
Data        datetime64[ns]
bola1              float64
bola2              float64
bola3              float64
bola4              float64
bola5              float64
bola6              float64
dtype: object

In [5]:
sena['Data do Sorteio'] = pd.to_datetime(sena['Data'], infer_datetime_format=True)

In [6]:
df = sena[(sena['Data do Sorteio']>='1996-02-28') ].copy()

In [7]:
df.head()

,Concurso,Data,bola1,bola2,bola3,bola4,bola5,bola6,Data do Sorteio
0,1,1996-03-11,41.0,5.0,4.0,52.0,30.0,33.0,1996-03-11
1,2,1996-03-18,9.0,39.0,37.0,49.0,43.0,41.0,1996-03-18
2,3,1996-03-25,36.0,30.0,10.0,11.0,29.0,47.0,1996-03-25
3,4,1996-04-01,6.0,59.0,42.0,27.0,1.0,5.0,1996-04-01
4,5,1996-04-08,1.0,19.0,46.0,6.0,16.0,2.0,1996-04-08


In [8]:
df.dtypes

Concurso                    int64
Data               datetime64[ns]
bola1                     float64
bola2                     float64
bola3                     float64
bola4                     float64
bola5                     float64
bola6                     float64
Data do Sorteio    datetime64[ns]
dtype: object

In [9]:
df.shape

(2846, 9)

In [10]:
#a = df[['Bola1','Bola2','Bola3','Bola4','Bola5','Bola6']].values
#a.sort(axis=1)  # no ascending argument
#df2 = pd.DataFrame(a, df.index, ['Bola1','Bola2','Bola3','Bola4','Bola5','Bola6'])
#df2.head()

In [11]:
df['dia']        = df['Data do Sorteio'].dt.day
df['mes']        = df['Data do Sorteio'].dt.month
df['ano']        = df['Data do Sorteio'].dt.year
df['semana_ano'] = df['Data do Sorteio'].dt.isocalendar().week
df['dia_semana'] = df['Data do Sorteio'].dt.dayofweek

In [12]:
from pyluach import dates, hebrewcal, parshios
from hebrew_numbers import gematria_to_int

#date = dates.GregorianDate(df['ano'], df['mes'], df['dia'] )
#parshios.getparsha(date)
#parshios.getparsha_string(date, israel=True)
#parshios.getparsha_string(date, hebrew=True)

In [13]:
for index, row in df[['ano','mes','dia']].iterrows():
    date = dates.GregorianDate(row['ano'], row['mes'], row['dia'] )
    #print(parshios.getparsha_string(date, hebrew=True))
    df.at[index, 'parashat'] = parshios.getparsha_string(date, hebrew=True) 


In [14]:
df.loc[ ( df['parashat'].isnull() ) ,'parashat'] = 'יום טוב'

In [15]:
df.head()

,Concurso,Data,bola1,bola2,bola3,bola4,bola5,bola6,Data do Sorteio,dia,mes,ano,semana_ano,dia_semana,parashat
0,1,1996-03-11,41.0,5.0,4.0,52.0,30.0,33.0,1996-03-11,11,3,1996,11,0,"ויקהל, פקודי"
1,2,1996-03-18,9.0,39.0,37.0,49.0,43.0,41.0,1996-03-18,18,3,1996,12,0,ויקרא
2,3,1996-03-25,36.0,30.0,10.0,11.0,29.0,47.0,1996-03-25,25,3,1996,13,0,צו
3,4,1996-04-01,6.0,59.0,42.0,27.0,1.0,5.0,1996-04-01,1,4,1996,14,0,יום טוב
4,5,1996-04-08,1.0,19.0,46.0,6.0,16.0,2.0,1996-04-08,8,4,1996,15,0,שמיני


In [16]:
from hebrew import Hebrew
from hebrew import GematriaTypes


hs = Hebrew('בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃')
print(hs.gematria())
print(hs.gematria(GematriaTypes.MISPAR_GADOL))
print(hs.gematria(GematriaTypes.MISPAR_HECHRACHI))
print(hs.gematria(GematriaTypes.MISPAR_SIDURI))
print(hs.gematria(GematriaTypes.MISPAR_KATAN))
print(hs.gematria(GematriaTypes.MISPAR_PERATI))
print(hs.gematria(GematriaTypes.ATBASH))
print(hs.gematria(GematriaTypes.ALBAM))
print(hs.gematria(GematriaTypes.MISPAR_MESHULASH))
print(hs.gematria(GematriaTypes.MISPAR_KIDMI))
print(hs.gematria(GematriaTypes.MISPAR_MISPARI))
print(hs.gematria(GematriaTypes.AYAK_BACHAR))
print(hs.gematria(GematriaTypes.AYAK_BAKAR))
print(hs.gematria(GematriaTypes.OFANIM))
print(hs.gematria(GematriaTypes.ACHAS_BETA))
print(hs.gematria(GematriaTypes.AVGAD))
print(hs.gematria(GematriaTypes.REVERSE_AVGAD))
print(hs.gematria(GematriaTypes.MISPAR_MUSAFI))
print(hs.gematria(GematriaTypes.MISPAR_BONEEH))
print(hs.gematria(GematriaTypes.MISPAR_HAMERUBAH_HAKLALI))
print(hs.gematria(GematriaTypes.MISPAR_BONEEH))
print(hs.gematria(GematriaTypes.MISPAR_HAMERUBAH_HAKLALI))
print(hs.gematria(GematriaTypes.MISPAR_HAACHOR))
print(hs.gematria(GematriaTypes.MISPAR_KATAN_MISPARI))
print(hs.gematria(GematriaTypes.MISPAR_KOLEL))
print(hs.gematria(GematriaTypes.MISPAR_SHEMI_MILUI))
print(hs.gematria(GematriaTypes.MISPAR_NEELAM))

2701
4631
2701
329
82
794225
3541
1571
270951613
10338
13256
1355
1355
2453
2372
2096
4236
2729
40035
7295401
40035
7295401
38294
1
2708
5446
2745


In [17]:
print(hs.gematria())  # 2701
print(hs.gematria(GematriaTypes.MISPAR_HECHRACHI))  # 4631

2701
2701


In [18]:
for index, row in df[['parashat']].iterrows():
    print(index,Hebrew(row['parashat']))
    df.at[index, 'gematria_MISPAR_HECHRACHI']          = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_HECHRACHI)
    df.at[index, 'gematria_MISPAR_GADOL']              = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_GADOL)
    df.at[index, 'gematria_MISPAR_SIDURI']             = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_SIDURI)
    df.at[index, 'gematria_MISPAR_KATAN']              = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_KATAN)
    df.at[index, 'gematria_MISPAR_PERATI']             = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_PERATI)
    df.at[index, 'gematria_ATBASH']                    = Hebrew(row['parashat']).gematria(GematriaTypes.ATBASH)
    df.at[index, 'gematria_ALBAM']                     = Hebrew(row['parashat']).gematria(GematriaTypes.ALBAM)
    df.at[index, 'gematria_MISPAR_MESHULASH']          = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_MESHULASH)
    df.at[index, 'gematria_MISPAR_KIDMI']              = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_KIDMI)
    df.at[index, 'gematria_MISPAR_MISPARI']            = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_MISPARI)
    df.at[index, 'gematria_AYAK_BACHAR']               = Hebrew(row['parashat']).gematria(GematriaTypes.AYAK_BACHAR)
    df.at[index, 'gematria_AYAK_BAKAR']                = Hebrew(row['parashat']).gematria(GematriaTypes.AYAK_BAKAR)
    df.at[index, 'gematria_OFANIM']                    = Hebrew(row['parashat']).gematria(GematriaTypes.OFANIM)
    df.at[index, 'gematria_ACHAS_BETA']                = Hebrew(row['parashat']).gematria(GematriaTypes.ACHAS_BETA)
    df.at[index, 'gematria_ACHAS_BETA']                = Hebrew(row['parashat']).gematria(GematriaTypes.ACHAS_BETA)
    df.at[index, 'gematria_AVGAD']                     = Hebrew(row['parashat']).gematria(GematriaTypes.AVGAD)
    df.at[index, 'gematria_REVERSE_AVGAD']             = Hebrew(row['parashat']).gematria(GematriaTypes.REVERSE_AVGAD)
    df.at[index, 'gematria_MISPAR_MUSAFI']             = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_MUSAFI)
    df.at[index, 'gematria_MISPAR_BONEEH']             = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_BONEEH)    
    df.at[index, 'gematria_MISPAR_HAMERUBAH_HAKLALI']  = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_HAMERUBAH_HAKLALI)
    df.at[index, 'gematria_MISPAR_HAACHOR']            = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_HAACHOR)
    df.at[index, 'gematria_MISPAR_KATAN_MISPARI']      = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_KATAN_MISPARI)
    df.at[index, 'gematria_MISPAR_SHEMI_MILUI']        = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_SHEMI_MILUI)
    df.at[index, 'gematria_MISPAR_NEELAM']             = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_NEELAM)

0 ויקהל, פקודי
1 ויקרא
2 צו
3 יום טוב
4 שמיני
5 תזריע, מצורע
6 אחרי מות, קדושים
7 אמור
8 בהר, בחוקותי
9 במדבר
10 יום טוב
11 נשא
12 בהעלותך
13 שלח
14 קרח
15 חקת, בלק
16 פינחס
17 מטות, מסעי
18 דברים
19 ואתחנן
20 עקב
21 ראה
22 שופטים
23 כי תצא
24 כי תבא
25 נצבים, וילך
26 יום טוב
27 האזינו
28 יום טוב
29 יום טוב
30 בראשית
31 נח
32 לך לך
33 וירא
34 חיי שרה
35 תולדות
36 ויצא
37 וישלח
38 וישב
39 מקץ
40 ויגש
41 ויחי
42 שמות
43 וארא
44 בא
45 בשלח
46 יתרו
47 משפטים
48 תרומה
49 תצוה
50 כי תשא
51 ויקהל
52 פקודי
53 ויקרא
54 צו
55 שמיני
56 תזריע
57 מצורע
58 יום טוב
59 אחרי מות
60 קדושים
61 אמור
62 בהר
63 בחוקותי
64 במדבר
65 נשא
66 בהעלותך
67 שלח
68 קרח
69 חקת
70 בלק
71 פינחס
72 מטות, מסעי
73 דברים
74 ואתחנן
75 עקב
76 ראה
77 שופטים
78 כי תצא
79 כי תבא
80 נצבים, וילך
81 האזינו
82 יום טוב
83 יום טוב
84 בראשית
85 נח
86 לך לך
87 וירא
88 חיי שרה
89 תולדות
90 ויצא
91 וישלח
92 וישב
93 מקץ
94 ויגש
95 ויחי
96 שמות
97 וארא
98 בא
99 בשלח
100 יתרו
101 משפטים
102 תרומה
103 תצוה
104 כי תשא
105 ויקהל, פקודי
106 ויקר

In [19]:
print(df.loc[ 215 , 'parashat'])

יום טוב


In [20]:
#df[df['parashat'].isnull() ].tail(11)
df.head()

,Concurso,Data,bola1,bola2,bola3,bola4,bola5,bola6,Data do Sorteio,dia,...,gematria_ACHAS_BETA,gematria_AVGAD,gematria_REVERSE_AVGAD,gematria_MISPAR_MUSAFI,gematria_MISPAR_BONEEH,gematria_MISPAR_HAMERUBAH_HAKLALI,gematria_MISPAR_HAACHOR,gematria_MISPAR_KATAN_MISPARI,gematria_MISPAR_SHEMI_MILUI,gematria_MISPAR_NEELAM
0,1,1996-03-11,41.0,5.0,4.0,52.0,30.0,33.0,1996-03-11,11,...,403.0,595.0,305.0,361.0,2001.0,123201.0,1860.0,9.0,1031.0,680.0
1,2,1996-03-18,9.0,39.0,37.0,49.0,43.0,41.0,1996-03-18,18,...,139.0,529.0,604.0,322.0,771.0,100489.0,1131.0,2.0,839.0,522.0
2,3,1996-03-25,36.0,30.0,10.0,11.0,29.0,47.0,1996-03-25,25,...,44.0,107.0,85.0,98.0,186.0,9216.0,102.0,6.0,116.0,20.0
3,4,1996-04-01,6.0,59.0,42.0,27.0,1.0,5.0,1996-04-01,1,...,439.0,97.0,58.0,79.0,291.0,5329.0,220.0,1.0,955.0,882.0
4,5,1996-04-08,1.0,19.0,46.0,6.0,16.0,2.0,1996-04-08,8,...,667.0,550.0,288.0,415.0,1800.0,168100.0,660.0,5.0,576.0,166.0


In [21]:
df.dtypes

Concurso                                      int64
Data                                 datetime64[ns]
bola1                                       float64
bola2                                       float64
bola3                                       float64
bola4                                       float64
bola5                                       float64
bola6                                       float64
Data do Sorteio                      datetime64[ns]
dia                                           int64
mes                                           int64
ano                                           int64
semana_ano                                   UInt32
dia_semana                                    int64
parashat                                     object
gematria_MISPAR_HECHRACHI                   float64
gematria_MISPAR_GADOL                       float64
gematria_MISPAR_SIDURI                      float64
gematria_MISPAR_KATAN                       float64
gematria_MIS

In [22]:
#https://pypi.org/project/hebrew/
df['bola1'] = df['bola1'].fillna(0.0).astype(np.int64)
df['bola2'] = df['bola2'].fillna(0.0).astype(np.int64)
df['bola3'] = df['bola3'].fillna(0.0).astype(np.int64)
df['bola4'] = df['bola4'].fillna(0.0).astype(np.int64)
df['bola5'] = df['bola5'].fillna(0.0).astype(np.int64)
df['bola6'] = df['bola6'].fillna(0.0).astype(np.int64)

df['gem_mega1'] = df['bola1']+df['bola2']+df['bola3']+df['bola4']+df['bola5']+df['bola6']
df['gem_mega2'] = df['bola1']*df['bola2']*df['bola3']*df['bola4']*df['bola5']*df['bola6']
df['gem_mega3'] = (df['bola1']**2)+(df['bola2']**2)+(df['bola3']**2)+(df['bola4']**2)+(df['bola5']**2)+(df['bola6']**2)

In [23]:
df['gem_mega1'] = df['gem_mega1'].fillna(0.0).astype(np.int64)
df['gem_mega2'] = df['gem_mega2'].fillna(0.0).astype(np.int64)
df['gem_mega3'] = df['gem_mega3'].fillna(0.0).astype(np.int64)

In [24]:
#list(str(df['gem_mega1']))
#sum([int(i) for i in str(df['gem_mega1'])])
df.dtypes

Concurso                                      int64
Data                                 datetime64[ns]
bola1                                         int64
bola2                                         int64
bola3                                         int64
bola4                                         int64
bola5                                         int64
bola6                                         int64
Data do Sorteio                      datetime64[ns]
dia                                           int64
mes                                           int64
ano                                           int64
semana_ano                                   UInt32
dia_semana                                    int64
parashat                                     object
gematria_MISPAR_HECHRACHI                   float64
gematria_MISPAR_GADOL                       float64
gematria_MISPAR_SIDURI                      float64
gematria_MISPAR_KATAN                       float64
gematria_MIS

In [25]:
#sum(list(str(row['gem_mega1'])))
df[['bola1','bola2','bola3','bola4','bola5','bola6','gem_mega2']][df['gem_mega2']<0].head()

,bola1,bola2,bola3,bola4,bola5,bola6,gem_mega2


In [26]:
#! pip install hebrew

for index, row in df[['gem_mega1','gem_mega2','gem_mega3']].iterrows():
    print(list(str(row['gem_mega3'])))    
    df.at[index, 'gem_mega1_reduz'] = sum([int(float(i)) for i in str(row['gem_mega1'])])
    df.at[index, 'gem_mega2_reduz'] = sum([int(float(i)) for i in str(row['gem_mega2'])])
    df.at[index, 'gem_mega3_reduz'] = sum([int(float(i)) for i in str(row['gem_mega3'])])

['6', '4', '1', '5']
['8', '9', '0', '2']
['5', '4', '6', '7']
['6', '0', '3', '6']
['2', '7', '7', '4']
['4', '8', '7', '2']
['5', '4', '5', '5']
['8', '1', '3', '6']
['1', '4', '5', '9', '0']
['6', '0', '9', '9']
['1', '0', '5', '0', '8']
['3', '6', '1', '1']
['1', '2', '1', '0', '9']
['6', '5', '3', '6']
['1', '1', '3', '6', '3']
['1', '1', '9', '4', '5']
['3', '6', '6', '7']
['8', '8', '2', '3']
['5', '0', '7', '0']
['6', '1', '2', '0']
['9', '7', '4', '7']
['7', '7', '3', '9']
['1', '1', '9', '6', '5']
['3', '9', '8', '3']
['1', '4', '0', '0', '2']
['1', '2', '5', '0', '6']
['6', '4', '8', '4']
['7', '6', '2', '1']
['8', '6', '1', '8']
['5', '2', '5', '5']
['8', '8', '9', '9']
['6', '3', '9', '1']
['7', '0', '8', '9']
['8', '3', '4', '1']
['7', '4', '0', '7']
['4', '5', '3', '4']
['8', '3', '1', '7']
['1', '1', '6', '7', '9']
['9', '6', '3', '4']
['4', '5', '6', '7']
['7', '2', '1', '9']
['3', '4', '9', '2']
['4', '9', '9', '4']
['6', '9', '5', '0']
['1', '0', '0', '8', '8']
['7',

In [27]:
from hebrew_numbers import gematria_to_int
import unicodedata
gematria_to_int('דברים')


816

In [28]:
df.head()

,Concurso,Data,bola1,bola2,bola3,bola4,bola5,bola6,Data do Sorteio,dia,...,gematria_MISPAR_HAACHOR,gematria_MISPAR_KATAN_MISPARI,gematria_MISPAR_SHEMI_MILUI,gematria_MISPAR_NEELAM,gem_mega1,gem_mega2,gem_mega3,gem_mega1_reduz,gem_mega2_reduz,gem_mega3_reduz
0,1,1996-03-11,41,5,4,52,30,33,1996-03-11,11,...,1860.0,9.0,1031.0,680.0,165,42213600,6415,12.0,18.0,16.0
1,2,1996-03-18,9,39,37,49,43,41,1996-03-18,18,...,1131.0,2.0,839.0,522.0,218,1121907969,8902,11.0,45.0,19.0
2,3,1996-03-25,36,30,10,11,29,47,1996-03-25,25,...,102.0,6.0,116.0,20.0,163,161924400,5467,10.0,27.0,22.0
3,4,1996-04-01,6,59,42,27,1,5,1996-04-01,1,...,220.0,1.0,955.0,882.0,140,2007180,6036,5.0,18.0,15.0
4,5,1996-04-08,1,19,46,6,16,2,1996-04-08,8,...,660.0,5.0,576.0,166.0,90,167808,2774,9.0,30.0,20.0


In [29]:
df.head()

,Concurso,Data,bola1,bola2,bola3,bola4,bola5,bola6,Data do Sorteio,dia,...,gematria_MISPAR_HAACHOR,gematria_MISPAR_KATAN_MISPARI,gematria_MISPAR_SHEMI_MILUI,gematria_MISPAR_NEELAM,gem_mega1,gem_mega2,gem_mega3,gem_mega1_reduz,gem_mega2_reduz,gem_mega3_reduz
0,1,1996-03-11,41,5,4,52,30,33,1996-03-11,11,...,1860.0,9.0,1031.0,680.0,165,42213600,6415,12.0,18.0,16.0
1,2,1996-03-18,9,39,37,49,43,41,1996-03-18,18,...,1131.0,2.0,839.0,522.0,218,1121907969,8902,11.0,45.0,19.0
2,3,1996-03-25,36,30,10,11,29,47,1996-03-25,25,...,102.0,6.0,116.0,20.0,163,161924400,5467,10.0,27.0,22.0
3,4,1996-04-01,6,59,42,27,1,5,1996-04-01,1,...,220.0,1.0,955.0,882.0,140,2007180,6036,5.0,18.0,15.0
4,5,1996-04-08,1,19,46,6,16,2,1996-04-08,8,...,660.0,5.0,576.0,166.0,90,167808,2774,9.0,30.0,20.0


In [30]:
# https://www.torahcalc.com/tools/gematria
for id in ['MISPAR_HECHRACHI','MISPAR_GADOL','MISPAR_SIDURI','MISPAR_KATAN','MISPAR_PERATI','ATBASH',
           'ALBAM','MISPAR_MESHULASH','MISPAR_KIDMI','MISPAR_MISPARI','AYAK_BACHAR','AYAK_BAKAR',
           'OFANIM','ACHAS_BETA','ACHAS_BETA','AVGAD','REVERSE_AVGAD','MISPAR_MUSAFI','MISPAR_BONEEH',
           'MISPAR_HAMERUBAH_HAKLALI','MISPAR_HAACHOR','MISPAR_KATAN_MISPARI','MISPAR_SHEMI_MILUI','MISPAR_NEELAM']:

           print('gem_mega1:', df[df['gematria_{0}'.format(id)]==df['gem_mega1_reduz'] ].shape[0] ,'- ',id ,'\n')
           print('gem_mega2:', df[df['gematria_{0}'.format(id)]==df['gem_mega2_reduz'] ].shape[0] ,'- ',id, '\n')
           print('gem_mega3:', df[df['gematria_{0}'.format(id)]==df['gem_mega3_reduz'] ].shape[0] ,'- ',id, '\n')

           df.loc[ (df['gematria_{0}'.format(id)]==df['gem_mega1_reduz']), 'match_mega1'] = '{0}'.format(id)
           df.loc[ (df['gematria_{0}'.format(id)]==df['gem_mega2_reduz']), 'match_mega2'] = '{0}'.format(id)
           df.loc[ (df['gematria_{0}'.format(id)]==df['gem_mega3_reduz']), 'match_mega3'] = '{0}'.format(id)

           df.loc[ (df['gematria_{0}'.format(id)]==df['gem_mega1']), 'match_1'] = '{0}'.format(id)
           df.loc[ (df['gematria_{0}'.format(id)]==df['gem_mega2']), 'match_2'] = '{0}'.format(id)
           df.loc[ (df['gematria_{0}'.format(id)]==df['gem_mega3']), 'match_3'] = '{0}'.format(id)


gem_mega1: 0 -  MISPAR_HECHRACHI 

gem_mega2: 1 -  MISPAR_HECHRACHI 

gem_mega3: 0 -  MISPAR_HECHRACHI 

gem_mega1: 0 -  MISPAR_GADOL 

gem_mega2: 1 -  MISPAR_GADOL 

gem_mega3: 0 -  MISPAR_GADOL 

gem_mega1: 0 -  MISPAR_SIDURI 

gem_mega2: 31 -  MISPAR_SIDURI 

gem_mega3: 11 -  MISPAR_SIDURI 

gem_mega1: 125 -  MISPAR_KATAN 

gem_mega2: 45 -  MISPAR_KATAN 

gem_mega3: 104 -  MISPAR_KATAN 

gem_mega1: 1 -  MISPAR_PERATI 

gem_mega2: 0 -  MISPAR_PERATI 

gem_mega3: 0 -  MISPAR_PERATI 

gem_mega1: 0 -  ATBASH 

gem_mega2: 0 -  ATBASH 

gem_mega3: 2 -  ATBASH 

gem_mega1: 1 -  ALBAM 

gem_mega2: 0 -  ALBAM 

gem_mega3: 1 -  ALBAM 

gem_mega1: 5 -  MISPAR_MESHULASH 

gem_mega2: 2 -  MISPAR_MESHULASH 

gem_mega3: 1 -  MISPAR_MESHULASH 

gem_mega1: 0 -  MISPAR_KIDMI 

gem_mega2: 0 -  MISPAR_KIDMI 

gem_mega3: 0 -  MISPAR_KIDMI 

gem_mega1: 0 -  MISPAR_MISPARI 

gem_mega2: 0 -  MISPAR_MISPARI 

gem_mega3: 0 -  MISPAR_MISPARI 

gem_mega1: 0 -  AYAK_BACHAR 

gem_mega2: 2 -  AYAK_BACHAR 

gem_me

In [31]:
df[~(df['match_mega1'].isnull()) & ~(df['match_1'].isnull())].head(15)

,Concurso,Data,bola1,bola2,bola3,bola4,bola5,bola6,Data do Sorteio,dia,...,gem_mega3,gem_mega1_reduz,gem_mega2_reduz,gem_mega3_reduz,match_mega1,match_mega2,match_mega3,match_1,match_2,match_3
263,264,2001-03-24,5,25,31,13,1,33,2001-03-24,24,...,2870,9.0,30.0,17.0,MISPAR_KATAN_MISPARI,NaN,NaN,MISPAR_SIDURI,NaN,NaN
391,392,2002-08-31,49,47,39,31,15,23,2002-08-31,31,...,7846,6.0,45.0,25.0,MISPAR_KATAN_MISPARI,NaN,NaN,REVERSE_AVGAD,NaN,NaN
419,420,2002-12-07,43,12,59,30,57,29,2002-12-07,7,...,10464,5.0,45.0,15.0,MISPAR_KATAN_MISPARI,NaN,NaN,MISPAR_HECHRACHI,NaN,NaN
628,629,2005-01-08,40,47,39,16,51,15,2005-01-08,8,...,8412,10.0,45.0,15.0,MISPAR_KATAN,NaN,NaN,MISPAR_GADOL,NaN,NaN
670,671,2005-06-08,27,16,39,51,9,38,2005-06-08,8,...,6632,9.0,45.0,17.0,MISPAR_KATAN_MISPARI,NaN,NaN,OFANIM,NaN,NaN
695,696,2005-09-03,37,16,34,57,53,9,2005-09-03,3,...,8920,8.0,36.0,19.0,MISPAR_KATAN_MISPARI,NaN,NaN,MISPAR_GADOL,NaN,NaN
978,979,2008-06-18,28,3,39,51,31,15,2008-06-18,18,...,6101,14.0,36.0,8.0,MISPAR_KATAN,NaN,NaN,ACHAS_BETA,NaN,NaN
1079,1080,2009-06-06,56,11,20,41,34,18,2009-06-06,6,...,6818,9.0,27.0,23.0,MISPAR_KATAN_MISPARI,NaN,NaN,OFANIM,NaN,NaN
1575,1576,2014-02-22,26,30,21,15,13,46,2014-02-22,22,...,4527,7.0,36.0,18.0,MISPAR_KATAN_MISPARI,NaN,NaN,MISPAR_GADOL,NaN,NaN
1708,1709,2015-05-30,30,7,47,35,19,42,2015-05-30,30,...,6508,9.0,36.0,19.0,MISPAR_KATAN_MISPARI,MISPAR_SIDURI,NaN,OFANIM,NaN,NaN


In [32]:
df[~(df['match_mega1'].isnull()) & ~(df['match_1'].isnull())].shape[0]

13

In [33]:
df.dtypes

Concurso                                      int64
Data                                 datetime64[ns]
bola1                                         int64
bola2                                         int64
bola3                                         int64
bola4                                         int64
bola5                                         int64
bola6                                         int64
Data do Sorteio                      datetime64[ns]
dia                                           int64
mes                                           int64
ano                                           int64
semana_ano                                   UInt32
dia_semana                                    int64
parashat                                     object
gematria_MISPAR_HECHRACHI                   float64
gematria_MISPAR_GADOL                       float64
gematria_MISPAR_SIDURI                      float64
gematria_MISPAR_KATAN                       float64
gematria_MIS

In [34]:
match1 = df[['Concurso','Data','bola1','bola2','bola3','bola4','bola5','bola6',
             'parashat','gem_mega1','gem_mega2','gem_mega3','gem_mega1_reduz',
             'gem_mega2_reduz','gem_mega3_reduz','match_mega1','match_mega2',
             'match_mega3','match_1','match_2','match_3']][~(df['match_mega1'].isnull()) & ~(df['match_1'].isnull())].copy()

match2 = df[['Concurso','Data','bola1','bola2','bola3','bola4','bola5','bola6',
             'parashat','gem_mega1','gem_mega2','gem_mega3','gem_mega1_reduz',
             'gem_mega2_reduz','gem_mega3_reduz','match_mega1','match_mega2',
             'match_mega3','match_1','match_2','match_3']][~(df['match_mega1'].isnull()) & ~(df['match_mega3'].isnull())].copy()

match3 = df[['Concurso','Data','bola1','bola2','bola3','bola4','bola5','bola6',
             'parashat','gem_mega1','gem_mega2','gem_mega3','gem_mega1_reduz',
             'gem_mega2_reduz','gem_mega3_reduz','match_mega1','match_mega2',
             'match_mega3','match_1','match_2','match_3']][~(df['match_mega2'].isnull()) & ~(df['match_mega3'].isnull())].copy()

In [35]:
match3.head()

,Concurso,Data,bola1,bola2,bola3,bola4,bola5,bola6,parashat,gem_mega1,...,gem_mega3,gem_mega1_reduz,gem_mega2_reduz,gem_mega3_reduz,match_mega1,match_mega2,match_mega3,match_1,match_2,match_3
329,330,2002-01-19,2,4,23,37,14,51,בא,131,...,4715,5.0,30.0,17.0,MISPAR_BONEEH,AYAK_BAKAR,ACHAS_BETA,NaN,NaN,NaN
342,343,2002-03-09,4,6,7,24,12,41,"ויקהל, פקודי",94,...,2502,13.0,36.0,9.0,NaN,MISPAR_KATAN,MISPAR_KATAN_MISPARI,NaN,NaN,NaN
1356,1357,2012-01-25,55,52,26,19,1,24,בא,177,...,7343,15.0,30.0,17.0,NaN,AYAK_BAKAR,ACHAS_BETA,NaN,NaN,NaN
2689,2690,2024-02-20,43,45,9,28,55,33,תצוה,213,...,8853,6.0,27.0,24.0,MISPAR_KATAN_MISPARI,MISPAR_NEELAM,MISPAR_KATAN,NaN,NaN,NaN


In [36]:
#match1.to_excel("match.xlsx", sheet_name='match1')
#match2.to_excel("match.xlsx", sheet_name='match2')
#match3.to_excel("match.xlsx", sheet_name='match3')

with pd.ExcelWriter('output.xlsx') as writer:
    match1.to_excel(writer, sheet_name='match1')
    match2.to_excel(writer, sheet_name='match2')
    match3.to_excel(writer, sheet_name='match3')

In [37]:
import calendar
import jewishcalendar

In [38]:
def getJewishMonthName(month, year):
  if month == 1:
    return "Nisan"
  elif month == 2:
    return "Iyyar"
  elif month == 3:
    return "Sivan"
  elif month == 4:
    return "Tammuz"
  elif month == 5:
    return "Av"
  elif month == 6:
    return "Elul"
  elif month == 7:
    return "Tishri"
  elif month == 8:
    return "Heshvan"
  elif month == 9:
    return "Kislev"
  elif month == 10:
    return "Teveth"
  elif month == 11:
    return "Shevat"
  elif month == 12:
    if jewishcalendar.hebrew_leap(year):
      return "Adar I"
    else:
      return "Adar"
    
  elif month == 13:
    return "Adar II"

In [39]:
absdate = jewishcalendar.gregorian_to_absdate(2023, 1, 1)
hebYear, hebMonth, hebDay = jewishcalendar.absdate_to_hebrew(absdate)

In [40]:
print(hebYear, hebMonth, hebDay)

5783 10 8


In [41]:
#df2['absdate'] = jewishcalendar.gregorian_to_absdate(df2['ano'], df2['mes'], df2['dia'])
#df2['hebYear'], df2['hebMonth'], df2['hebDay'] = jewishcalendar.absdate_to_hebrew(df2['absdate'] )
#df2['absdate'] = df2[['ano','mes','dia']].applymap(jewishcalendar.gregorian_to_absdate, args=(1,2) ) 

In [42]:
hebYear = [] 
hebMonth = [] 
hebDay = [] 

for index, row in df[['ano','mes','dia']].iterrows():
    valor = jewishcalendar.gregorian_to_absdate(row['ano'],row['mes'],row['dia'])
    df.at[index, 'hebYear'], df.at[index, 'hebMonth'] , df.at[index, 'hebDay'] = jewishcalendar.absdate_to_hebrew(valor)


In [43]:
#df1 = pd.read_csv('hebcal_5784h_usa.csv')
parashat = []
for ano in [5776,5777,5778,5779,5780,5781,5782,5783,5784,5785,5786,5787,5788]:
    para = pd.read_csv('hebcal_{0}h_usa.csv'.format(ano))
    para.drop(columns=['Start Time','End Date','End Time','All day event','Description','Show time as','Location'] , inplace=True)
    parashat.append(para)

In [44]:
parashat[0].tail()

,Subject,Start Date
48,Parashat Re’eh,9/3/2016
49,Parashat Shoftim,9/10/2016
50,Parashat Ki Teitzei,9/17/2016
51,Parashat Ki Tavo,9/24/2016
52,Parashat Nitzavim,10/1/2016


In [45]:
import pandas as pd 
df_parashat =  pd.concat(parashat)

In [46]:
df_parashat.tail()

,Subject,Start Date
482,Rosh Chodesh Elul,8/7/2032
483,Rosh Hashana LaBehemot,8/8/2032
484,Rosh Chodesh Elul,8/8/2032
485,Leil Selichot,8/28/2032
486,Erev Rosh Hashana,9/5/2032


In [47]:
#! pip install python-hebrew-numbers

In [48]:
from hebrew_numbers import gematria_to_int
gematria_to_int(u'רח"צ')

298

In [49]:
from hebrew_numbers import int_to_gematria

int_to_gematria(298, gershayim=False)

'רחצ'

In [50]:
#! pip install pyluach
# https://pypi.org/project/pyluach/
from pyluach import dates, hebrewcal, parshios
today = dates.HebrewDate.today()

In [51]:
for month in hebrewcal.Year(5774).itermonths():
    print(month.month_name())

Tishrei
Cheshvan
Kislev
Teves
Shevat
Adar 1
Adar 2
Nissan
Iyar
Sivan
Tammuz
Av
Elul


In [52]:
date = dates.GregorianDate(2010, 10, 6)
parshios.getparsha(date)
parshios.getparsha_string(date, israel=True)
parshios.getparsha_string(date, hebrew=True)

'נח'

In [53]:
parshios.getparsha_string(date, hebrew=True)

'נח'

In [54]:
new_date = dates.GregorianDate(2021, 3, 10)

In [55]:
parshios.getparsha_string(new_date, hebrew=True)

'ויקהל, פקודי'

In [56]:
import itertools
itertools.product(range(1, 20), repeat=1)


In [116]:
import datetime
import pandas as pd
from hebrew import Hebrew
from hebrew import GematriaTypes
from pyluach import dates, hebrewcal, parshios
from hebrew_numbers import gematria_to_int
import calendar
import jewishcalendar
 
start = datetime.date(2024,12,20)
 
# initializing K 
K = 376
 
res = []
 
for day in range(K):
    date = (start + datetime.timedelta(days = day)).isoformat()
    res.append(date)
     
# printing result
print("Next K dates list: " + str(res))


df_atid = pd.DataFrame({'Data':res})


Next K dates list: ['2024-12-20', '2024-12-21', '2024-12-22', '2024-12-23', '2024-12-24', '2024-12-25', '2024-12-26', '2024-12-27', '2024-12-28', '2024-12-29', '2024-12-30', '2024-12-31', '2025-01-01', '2025-01-02', '2025-01-03', '2025-01-04', '2025-01-05', '2025-01-06', '2025-01-07', '2025-01-08', '2025-01-09', '2025-01-10', '2025-01-11', '2025-01-12', '2025-01-13', '2025-01-14', '2025-01-15', '2025-01-16', '2025-01-17', '2025-01-18', '2025-01-19', '2025-01-20', '2025-01-21', '2025-01-22', '2025-01-23', '2025-01-24', '2025-01-25', '2025-01-26', '2025-01-27', '2025-01-28', '2025-01-29', '2025-01-30', '2025-01-31', '2025-02-01', '2025-02-02', '2025-02-03', '2025-02-04', '2025-02-05', '2025-02-06', '2025-02-07', '2025-02-08', '2025-02-09', '2025-02-10', '2025-02-11', '2025-02-12', '2025-02-13', '2025-02-14', '2025-02-15', '2025-02-16', '2025-02-17', '2025-02-18', '2025-02-19', '2025-02-20', '2025-02-21', '2025-02-22', '2025-02-23', '2025-02-24', '2025-02-25', '2025-02-26', '2025-02-27', 

In [117]:
df_atid.head()

,Data
0,2024-12-20
1,2024-12-21
2,2024-12-22
3,2024-12-23
4,2024-12-24


In [118]:
df_atid['Data']       = pd.to_datetime(df_atid['Data'], exact=True, cache=True, format='%Y-%m-%d')
df_atid['dia']        = df_atid['Data'].dt.day
df_atid['dia']        = df_atid['Data'].dt.day
df_atid['mes']        = df_atid['Data'].dt.month
df_atid['ano']        = df_atid['Data'].dt.year
df_atid['semana_ano'] = df_atid['Data'].dt.isocalendar().week
df_atid['dia_semana'] = df_atid['Data'].dt.dayofweek


for index, row in df_atid[['ano','mes','dia']].iterrows():
    date = dates.GregorianDate(row['ano'], row['mes'], row['dia'] )
    print(parshios.getparsha_string(date, hebrew=True))
    df_atid.at[index, 'parashat'] = parshios.getparsha_string(date, hebrew=True) 

וישב
וישב
מקץ
מקץ
מקץ
מקץ
מקץ
מקץ
מקץ
ויגש
ויגש
ויגש
ויגש
ויגש
ויגש
ויגש
ויחי
ויחי
ויחי
ויחי
ויחי
ויחי
ויחי
שמות
שמות
שמות
שמות
שמות
שמות
שמות
וארא
וארא
וארא
וארא
וארא
וארא
וארא
בא
בא
בא
בא
בא
בא
בא
בשלח
בשלח
בשלח
בשלח
בשלח
בשלח
בשלח
יתרו
יתרו
יתרו
יתרו
יתרו
יתרו
יתרו
משפטים
משפטים
משפטים
משפטים
משפטים
משפטים
משפטים
תרומה
תרומה
תרומה
תרומה
תרומה
תרומה
תרומה
תצוה
תצוה
תצוה
תצוה
תצוה
תצוה
תצוה
כי תשא
כי תשא
כי תשא
כי תשא
כי תשא
כי תשא
כי תשא
ויקהל
ויקהל
ויקהל
ויקהל
ויקהל
ויקהל
ויקהל
פקודי
פקודי
פקודי
פקודי
פקודי
פקודי
פקודי
ויקרא
ויקרא
ויקרא
ויקרא
ויקרא
ויקרא
ויקרא
צו
צו
צו
צו
צו
צו
צו
None
None
None
None
None
None
None
שמיני
שמיני
שמיני
שמיני
שמיני
שמיני
שמיני
תזריע, מצורע
תזריע, מצורע
תזריע, מצורע
תזריע, מצורע
תזריע, מצורע
תזריע, מצורע
תזריע, מצורע
אחרי מות, קדושים
אחרי מות, קדושים
אחרי מות, קדושים
אחרי מות, קדושים
אחרי מות, קדושים
אחרי מות, קדושים
אחרי מות, קדושים
אמור
אמור
אמור
אמור
אמור
אמור
אמור
בהר, בחוקותי
בהר, בחוקותי
בהר, בחוקותי
בהר, בחוקותי
בהר, בחוקותי
בהר, בחוקותי
בהר, בחוק

In [119]:
hebYear = [] 
hebMonth = [] 
hebDay = [] 

for index, row in df_atid[['ano','mes','dia']].iterrows():
    valor = jewishcalendar.gregorian_to_absdate(row['ano'],row['mes'],row['dia'])
    df_atid.at[index, 'hebYear'], df_atid.at[index, 'hebMonth'] , df_atid.at[index, 'hebDay'] = jewishcalendar.absdate_to_hebrew(valor)

In [120]:
df_atid.head()

,Data,dia,mes,ano,semana_ano,dia_semana,parashat,hebYear,hebMonth,hebDay
0,2024-12-20,20,12,2024,51,4,וישב,5785.0,9.0,19.0
1,2024-12-21,21,12,2024,51,5,וישב,5785.0,9.0,20.0
2,2024-12-22,22,12,2024,51,6,מקץ,5785.0,9.0,21.0
3,2024-12-23,23,12,2024,52,0,מקץ,5785.0,9.0,22.0
4,2024-12-24,24,12,2024,52,1,מקץ,5785.0,9.0,23.0


In [121]:
df_atid.loc[ ( df_atid['parashat'].isnull() ) ,'parashat'] = 'יום טוב'

In [122]:
for index, row in df_atid[['parashat']].iterrows():
    print(index,Hebrew(row['parashat']))
    df_atid.at[index, 'gematria_MISPAR_HECHRACHI']         = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_HECHRACHI)
    df_atid.at[index, 'gematria_MISPAR_GADOL']             = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_GADOL)
    df_atid.at[index, 'gematria_MISPAR_SIDURI']            = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_SIDURI)
    df_atid.at[index, 'gematria_MISPAR_KATAN']             = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_KATAN)
    df_atid.at[index, 'gematria_MISPAR_PERATI']            = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_PERATI)
    df_atid.at[index, 'gematria_ATBASH']                   = Hebrew(row['parashat']).gematria(GematriaTypes.ATBASH)
    df_atid.at[index, 'gematria_ALBAM']                    = Hebrew(row['parashat']).gematria(GematriaTypes.ALBAM)
    df_atid.at[index, 'gematria_MISPAR_MESHULASH']         = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_MESHULASH)
    df_atid.at[index, 'gematria_MISPAR_KIDMI']             = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_KIDMI)
    df_atid.at[index, 'gematria_MISPAR_MISPARI']           = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_MISPARI)
    df_atid.at[index, 'gematria_AYAK_BACHAR']              = Hebrew(row['parashat']).gematria(GematriaTypes.AYAK_BACHAR)
    df_atid.at[index, 'gematria_AYAK_BAKAR']               = Hebrew(row['parashat']).gematria(GematriaTypes.AYAK_BAKAR)
    df_atid.at[index, 'gematria_OFANIM']                   = Hebrew(row['parashat']).gematria(GematriaTypes.OFANIM)
    df_atid.at[index, 'gematria_ACHAS_BETA']               = Hebrew(row['parashat']).gematria(GematriaTypes.ACHAS_BETA)
    df_atid.at[index, 'gematria_ACHAS_BETA']               = Hebrew(row['parashat']).gematria(GematriaTypes.ACHAS_BETA)
    df_atid.at[index, 'gematria_AVGAD']                    = Hebrew(row['parashat']).gematria(GematriaTypes.AVGAD)
    df_atid.at[index, 'gematria_REVERSE_AVGAD']            = Hebrew(row['parashat']).gematria(GematriaTypes.REVERSE_AVGAD)
    df_atid.at[index, 'gematria_MISPAR_MUSAFI']            = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_MUSAFI)
    df_atid.at[index, 'gematria_MISPAR_BONEEH']            = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_BONEEH)    
    df_atid.at[index, 'gematria_MISPAR_HAMERUBAH_HAKLALI'] = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_HAMERUBAH_HAKLALI)
    df_atid.at[index, 'gematria_MISPAR_HAACHOR']           = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_HAACHOR)
    df_atid.at[index, 'gematria_MISPAR_KATAN_MISPARI']     = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_KATAN_MISPARI)
    df_atid.at[index, 'gematria_MISPAR_SHEMI_MILUI']       = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_SHEMI_MILUI)
    df_atid.at[index, 'gematria_MISPAR_NEELAM']            = Hebrew(row['parashat']).gematria(GematriaTypes.MISPAR_NEELAM)

0 וישב
1 וישב
2 מקץ
3 מקץ
4 מקץ
5 מקץ
6 מקץ
7 מקץ
8 מקץ
9 ויגש
10 ויגש
11 ויגש
12 ויגש
13 ויגש
14 ויגש
15 ויגש
16 ויחי
17 ויחי
18 ויחי
19 ויחי
20 ויחי
21 ויחי
22 ויחי
23 שמות
24 שמות
25 שמות
26 שמות
27 שמות
28 שמות
29 שמות
30 וארא
31 וארא
32 וארא
33 וארא
34 וארא
35 וארא
36 וארא
37 בא
38 בא
39 בא
40 בא
41 בא
42 בא
43 בא
44 בשלח
45 בשלח
46 בשלח
47 בשלח
48 בשלח
49 בשלח
50 בשלח
51 יתרו
52 יתרו
53 יתרו
54 יתרו
55 יתרו
56 יתרו
57 יתרו
58 משפטים
59 משפטים
60 משפטים
61 משפטים
62 משפטים
63 משפטים
64 משפטים
65 תרומה
66 תרומה
67 תרומה
68 תרומה
69 תרומה
70 תרומה
71 תרומה
72 תצוה
73 תצוה
74 תצוה
75 תצוה
76 תצוה
77 תצוה
78 תצוה
79 כי תשא
80 כי תשא
81 כי תשא
82 כי תשא
83 כי תשא
84 כי תשא
85 כי תשא
86 ויקהל
87 ויקהל
88 ויקהל
89 ויקהל
90 ויקהל
91 ויקהל
92 ויקהל
93 פקודי
94 פקודי
95 פקודי
96 פקודי
97 פקודי
98 פקודי
99 פקודי
100 ויקרא
101 ויקרא
102 ויקרא
103 ויקרא
104 ויקרא
105 ויקרא
106 ויקרא
107 צו
108 צו
109 צו
110 צו
111 צו
112 צו
113 צו
114 יום טוב
115 יום טוב
116 יום טוב
117 יום טוב
118 יום טוב
119

In [123]:
df_atid[df_atid['parashat']=='צו'][['Data','parashat','gematria_ACHAS_BETA','gematria_AVGAD','gematria_MISPAR_HAACHOR','gematria_MISPAR_NEELAM']].head(15)

,Data,parashat,gematria_ACHAS_BETA,gematria_AVGAD,gematria_MISPAR_HAACHOR,gematria_MISPAR_NEELAM
107,2025-04-06,צו,44.0,107.0,102.0,20.0
108,2025-04-07,צו,44.0,107.0,102.0,20.0
109,2025-04-08,צו,44.0,107.0,102.0,20.0
110,2025-04-09,צו,44.0,107.0,102.0,20.0
111,2025-04-10,צו,44.0,107.0,102.0,20.0
112,2025-04-11,צו,44.0,107.0,102.0,20.0
113,2025-04-12,צו,44.0,107.0,102.0,20.0


In [124]:
df_atid['controle'] = 1

In [125]:
for index, row in df_atid[['parashat']].iterrows():
    print(index,(row['parashat']))

0 וישב
1 וישב
2 מקץ
3 מקץ
4 מקץ
5 מקץ
6 מקץ
7 מקץ
8 מקץ
9 ויגש
10 ויגש
11 ויגש
12 ויגש
13 ויגש
14 ויגש
15 ויגש
16 ויחי
17 ויחי
18 ויחי
19 ויחי
20 ויחי
21 ויחי
22 ויחי
23 שמות
24 שמות
25 שמות
26 שמות
27 שמות
28 שמות
29 שמות
30 וארא
31 וארא
32 וארא
33 וארא
34 וארא
35 וארא
36 וארא
37 בא
38 בא
39 בא
40 בא
41 בא
42 בא
43 בא
44 בשלח
45 בשלח
46 בשלח
47 בשלח
48 בשלח
49 בשלח
50 בשלח
51 יתרו
52 יתרו
53 יתרו
54 יתרו
55 יתרו
56 יתרו
57 יתרו
58 משפטים
59 משפטים
60 משפטים
61 משפטים
62 משפטים
63 משפטים
64 משפטים
65 תרומה
66 תרומה
67 תרומה
68 תרומה
69 תרומה
70 תרומה
71 תרומה
72 תצוה
73 תצוה
74 תצוה
75 תצוה
76 תצוה
77 תצוה
78 תצוה
79 כי תשא
80 כי תשא
81 כי תשא
82 כי תשא
83 כי תשא
84 כי תשא
85 כי תשא
86 ויקהל
87 ויקהל
88 ויקהל
89 ויקהל
90 ויקהל
91 ויקהל
92 ויקהל
93 פקודי
94 פקודי
95 פקודי
96 פקודי
97 פקודי
98 פקודי
99 פקודי
100 ויקרא
101 ויקרא
102 ויקרא
103 ויקרא
104 ויקרא
105 ויקרא
106 ויקרא
107 צו
108 צו
109 צו
110 צו
111 צו
112 צו
113 צו
114 יום טוב
115 יום טוב
116 יום טוב
117 יום טוב
118 יום טוב
119

# Novos Concursos

In [126]:
sena.drop_duplicates( subset=['bola1','bola2','bola3','bola4','bola5','bola6'], keep='first', inplace=True)

In [131]:
#sena.drop( columns=['Data','Concurso','Data do Sorteio'], axis=1, inplace=True)

In [132]:
sena.shape

(2804, 13)

In [133]:
sena.head()

,bola1,bola2,bola3,bola4,bola5,bola6,gem_mega1,gem_mega2,gem_mega3,controle,gem_mega1_reduz,gem_mega2_reduz,gem_mega3_reduz
0,41,5,4,52,30,33,165,42213600,6415,1,12.0,18.0,16.0
1,9,39,37,49,43,41,218,1121907969,8902,1,11.0,45.0,19.0
2,36,30,10,11,29,47,163,161924400,5467,1,10.0,27.0,22.0
3,6,59,42,27,1,5,140,2007180,6036,1,5.0,18.0,15.0
4,1,19,46,6,16,2,90,167808,2774,1,9.0,30.0,20.0


In [134]:
sena['bola1'] = sena['bola1'].fillna(0.0).astype(np.int64)
sena['bola2'] = sena['bola2'].fillna(0.0).astype(np.int64)
sena['bola3'] = sena['bola3'].fillna(0.0).astype(np.int64)
sena['bola4'] = sena['bola4'].fillna(0.0).astype(np.int64)
sena['bola5'] = sena['bola5'].fillna(0.0).astype(np.int64)
sena['bola6'] = sena['bola6'].fillna(0.0).astype(np.int64)

sena['gem_mega1'] = sena['bola1']+ sena['bola2']+ sena['bola3']+ sena['bola4']+sena['bola5']+sena['bola6']
sena['gem_mega2'] = sena['bola1']* sena['bola2']* sena['bola3']* sena['bola4']* sena['bola5']* sena['bola6']
sena['gem_mega3'] = (sena['bola1']**2) + (sena['bola2']**2) + (sena['bola3']**2) + (sena['bola4']**2) + (sena['bola5']**2) + (sena['bola6']**2)

sena['controle'] = 1

In [135]:

for index, row in sena[['gem_mega1','gem_mega2','gem_mega3']].iterrows():
    print(list(str(row['gem_mega3'])))    
    sena.at[index, 'gem_mega1_reduz'] = sum([int(float(i)) for i in str(row['gem_mega1'])])
    sena.at[index, 'gem_mega2_reduz'] = sum([int(float(i)) for i in str(row['gem_mega2'])])
    sena.at[index, 'gem_mega3_reduz'] = sum([int(float(i)) for i in str(row['gem_mega3'])])

['6', '4', '1', '5']
['8', '9', '0', '2']
['5', '4', '6', '7']
['6', '0', '3', '6']
['2', '7', '7', '4']
['4', '8', '7', '2']
['5', '4', '5', '5']
['8', '1', '3', '6']
['1', '4', '5', '9', '0']
['6', '0', '9', '9']
['1', '0', '5', '0', '8']
['3', '6', '1', '1']
['1', '2', '1', '0', '9']
['6', '5', '3', '6']
['1', '1', '3', '6', '3']
['1', '1', '9', '4', '5']
['3', '6', '6', '7']
['8', '8', '2', '3']
['5', '0', '7', '0']
['6', '1', '2', '0']
['9', '7', '4', '7']
['7', '7', '3', '9']
['1', '1', '9', '6', '5']
['3', '9', '8', '3']
['1', '4', '0', '0', '2']
['1', '2', '5', '0', '6']
['6', '4', '8', '4']
['7', '6', '2', '1']
['8', '6', '1', '8']
['5', '2', '5', '5']
['8', '8', '9', '9']
['6', '3', '9', '1']
['7', '0', '8', '9']
['8', '3', '4', '1']
['7', '4', '0', '7']
['4', '5', '3', '4']
['8', '3', '1', '7']
['1', '1', '6', '7', '9']
['9', '6', '3', '4']
['4', '5', '6', '7']
['7', '2', '1', '9']
['3', '4', '9', '2']
['4', '9', '9', '4']
['6', '9', '5', '0']
['1', '0', '0', '8', '8']
['7',

In [136]:
merged = pd.merge(sena, df_atid, on='controle',how='outer')


In [137]:
merged.head()

,bola1,bola2,bola3,bola4,bola5,bola6,gem_mega1,gem_mega2,gem_mega3,controle,...,gematria_ACHAS_BETA,gematria_AVGAD,gematria_REVERSE_AVGAD,gematria_MISPAR_MUSAFI,gematria_MISPAR_BONEEH,gematria_MISPAR_HAMERUBAH_HAKLALI,gematria_MISPAR_HAACHOR,gematria_MISPAR_KATAN_MISPARI,gematria_MISPAR_SHEMI_MILUI,gematria_MISPAR_NEELAM
0,41,5,4,52,30,33,165,42213600,6415,1,...,136.0,430.0,215.0,322.0,656.0,101124.0,934.0,3.0,794.0,476.0
1,41,5,4,52,30,33,165,42213600,6415,1,...,136.0,430.0,215.0,322.0,656.0,101124.0,934.0,3.0,794.0,476.0
2,41,5,4,52,30,33,165,42213600,6415,1,...,209.0,350.0,200.0,233.0,410.0,52900.0,510.0,5.0,370.0,140.0
3,41,5,4,52,30,33,165,42213600,6415,1,...,209.0,350.0,200.0,233.0,410.0,52900.0,510.0,5.0,370.0,140.0
4,41,5,4,52,30,33,165,42213600,6415,1,...,209.0,350.0,200.0,233.0,410.0,52900.0,510.0,5.0,370.0,140.0


In [138]:

# https://www.torahcalc.com/tools/gematria
for id in ['MISPAR_HECHRACHI','MISPAR_GADOL','MISPAR_SIDURI','MISPAR_KATAN','MISPAR_PERATI','ATBASH',
           'ALBAM','MISPAR_MESHULASH','MISPAR_KIDMI','MISPAR_MISPARI','AYAK_BACHAR','AYAK_BAKAR',
           'OFANIM','ACHAS_BETA','ACHAS_BETA','AVGAD','REVERSE_AVGAD','MISPAR_MUSAFI','MISPAR_BONEEH',
           'MISPAR_HAMERUBAH_HAKLALI','MISPAR_HAACHOR','MISPAR_KATAN_MISPARI','MISPAR_SHEMI_MILUI','MISPAR_NEELAM']:

           print('gem_mega1:', merged[merged['gematria_{0}'.format(id)]==merged['gem_mega1_reduz'] ].shape[0] ,'- ',id ,'\n')
           print('gem_mega2:', merged[merged['gematria_{0}'.format(id)]==merged['gem_mega2_reduz'] ].shape[0] ,'- ',id, '\n')
           print('gem_mega3:', merged[merged['gematria_{0}'.format(id)]==merged['gem_mega3_reduz'] ].shape[0] ,'- ',id, '\n')

           merged.loc[ (merged['gematria_{0}'.format(id)]==merged['gem_mega1_reduz']), 'match_mega1'] = '{0}'.format(id)
           merged.loc[ (merged['gematria_{0}'.format(id)]==merged['gem_mega2_reduz']), 'match_mega2'] = '{0}'.format(id)
           merged.loc[ (merged['gematria_{0}'.format(id)]==merged['gem_mega3_reduz']), 'match_mega3'] = '{0}'.format(id)

           merged.loc[ (merged['gematria_{0}'.format(id)]==merged['gem_mega1']), 'match_1'] = '{0}'.format(id)
           merged.loc[ (merged['gematria_{0}'.format(id)]==merged['gem_mega2']), 'match_2'] = '{0}'.format(id)
           merged.loc[ (merged['gematria_{0}'.format(id)]==merged['gem_mega3']), 'match_3'] = '{0}'.format(id)

gem_mega1: 434 -  MISPAR_HECHRACHI 

gem_mega2: 120 -  MISPAR_HECHRACHI 

gem_mega3: 68 -  MISPAR_HECHRACHI 

gem_mega1: 434 -  MISPAR_GADOL 

gem_mega2: 120 -  MISPAR_GADOL 

gem_mega3: 68 -  MISPAR_GADOL 

gem_mega1: 434 -  MISPAR_SIDURI 

gem_mega2: 8458 -  MISPAR_SIDURI 

gem_mega3: 3463 -  MISPAR_SIDURI 

gem_mega1: 48969 -  MISPAR_KATAN 

gem_mega2: 9674 -  MISPAR_KATAN 

gem_mega3: 37411 -  MISPAR_KATAN 

gem_mega1: 791 -  MISPAR_PERATI 

gem_mega2: 0 -  MISPAR_PERATI 

gem_mega3: 42 -  MISPAR_PERATI 

gem_mega1: 336 -  ATBASH 

gem_mega2: 112 -  ATBASH 

gem_mega3: 2800 -  ATBASH 

gem_mega1: 1204 -  ALBAM 

gem_mega2: 63 -  ALBAM 

gem_mega3: 2688 -  ALBAM 

gem_mega1: 1666 -  MISPAR_MESHULASH 

gem_mega2: 98 -  MISPAR_MESHULASH 

gem_mega3: 252 -  MISPAR_MESHULASH 

gem_mega1: 728 -  MISPAR_KIDMI 

gem_mega2: 7 -  MISPAR_KIDMI 

gem_mega3: 14 -  MISPAR_KIDMI 

gem_mega1: 0 -  MISPAR_MISPARI 

gem_mega2: 0 -  MISPAR_MISPARI 

gem_mega3: 0 -  MISPAR_MISPARI 

gem_mega1: 0 -  AY

In [139]:
merged.head()

,bola1,bola2,bola3,bola4,bola5,bola6,gem_mega1,gem_mega2,gem_mega3,controle,...,gematria_MISPAR_HAACHOR,gematria_MISPAR_KATAN_MISPARI,gematria_MISPAR_SHEMI_MILUI,gematria_MISPAR_NEELAM,match_mega1,match_mega2,match_mega3,match_1,match_2,match_3
0,41,5,4,52,30,33,165,42213600,6415,1,...,934.0,3.0,794.0,476.0,MISPAR_KATAN,NaN,NaN,NaN,NaN,NaN
1,41,5,4,52,30,33,165,42213600,6415,1,...,934.0,3.0,794.0,476.0,MISPAR_KATAN,NaN,NaN,NaN,NaN,NaN
2,41,5,4,52,30,33,165,42213600,6415,1,...,510.0,5.0,370.0,140.0,NaN,NaN,NaN,NaN,NaN,NaN
3,41,5,4,52,30,33,165,42213600,6415,1,...,510.0,5.0,370.0,140.0,NaN,NaN,NaN,NaN,NaN,NaN
4,41,5,4,52,30,33,165,42213600,6415,1,...,510.0,5.0,370.0,140.0,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
merged = merged[ (merged['bola1']!=merged['bola2']) & (merged['bola1']!=merged['bola3']) & (merged['bola1']!=merged['bola4']) &
                 (merged['bola1']!=merged['bola5']) & (merged['bola1']!=merged['bola6']) & (merged['bola2']!=merged['bola3']) &
                 (merged['bola2']!=merged['bola4']) & (merged['bola2']!=merged['bola5']) & (merged['bola2']!=merged['bola6']) &
                 (merged['bola3']!=merged['bola4']) & (merged['bola3']!=merged['bola5']) & (merged['bola3']!=merged['bola6']) &
                 (merged['bola4']!=merged['bola5']) & (merged['bola4']!=merged['bola6']) ].copy()

In [141]:
new_match1 = merged[['Data','dia_semana','bola1','bola2','bola3','bola4','bola5','bola6',
             'parashat','gem_mega1','gem_mega2','gem_mega3','gem_mega1_reduz',
             'gem_mega2_reduz','gem_mega3_reduz','match_mega1','match_mega2',
             'match_mega3','match_1','match_2','match_3']][~(merged['match_mega1'].isnull()) & ~(merged['match_1'].isnull())].copy()

new_match2 = merged[['Data','dia_semana','bola1','bola2','bola3','bola4','bola5','bola6',
             'parashat','gem_mega1','gem_mega2','gem_mega3','gem_mega1_reduz',
             'gem_mega2_reduz','gem_mega3_reduz','match_mega1','match_mega2',
             'match_mega3','match_1','match_2','match_3']][~(merged['match_mega1'].isnull()) & ~(merged['match_mega3'].isnull())].copy()

new_match3 = merged[['Data','dia_semana','bola1','bola2','bola3','bola4','bola5','bola6',
             'parashat','gem_mega1','gem_mega2','gem_mega3','gem_mega1_reduz',
             'gem_mega2_reduz','gem_mega3_reduz','match_mega1','match_mega2',
             'match_mega3','match_1','match_2','match_3']][~(merged['match_mega2'].isnull()) & ~(merged['match_mega3'].isnull())].copy()

In [142]:
new_match3.head()

,Data,dia_semana,bola1,bola2,bola3,bola4,bola5,bola6,parashat,gem_mega1,...,gem_mega3,gem_mega1_reduz,gem_mega2_reduz,gem_mega3_reduz,match_mega1,match_mega2,match_mega3,match_1,match_2,match_3
5747,2025-04-06,6,60,58,49,32,20,34,צו,253,...,11945,10.0,24.0,20.0,NaN,MISPAR_SIDURI,MISPAR_NEELAM,NaN,NaN,NaN
5748,2025-04-07,0,60,58,49,32,20,34,צו,253,...,11945,10.0,24.0,20.0,NaN,MISPAR_SIDURI,MISPAR_NEELAM,NaN,NaN,NaN
5749,2025-04-08,1,60,58,49,32,20,34,צו,253,...,11945,10.0,24.0,20.0,NaN,MISPAR_SIDURI,MISPAR_NEELAM,NaN,NaN,NaN
5750,2025-04-09,2,60,58,49,32,20,34,צו,253,...,11945,10.0,24.0,20.0,NaN,MISPAR_SIDURI,MISPAR_NEELAM,NaN,NaN,NaN
5751,2025-04-10,3,60,58,49,32,20,34,צו,253,...,11945,10.0,24.0,20.0,NaN,MISPAR_SIDURI,MISPAR_NEELAM,NaN,NaN,NaN


In [143]:
#match1.to_excel("match.xlsx", sheet_name='match1')
#match2.to_excel("match.xlsx", sheet_name='match2')
#match3.to_excel("match.xlsx", sheet_name='match3')

with pd.ExcelWriter('output_new.xlsx') as writer:
    new_match3.to_excel(writer, sheet_name='match1')
    new_match3.to_excel(writer, sheet_name='match2')
    new_match3.to_excel(writer, sheet_name='match3')

In [161]:
dec = merged[['Data','parashat','bola1','bola2','bola3','bola4','bola5','bola6','match_mega1','match_mega2','match_mega3']][(merged['ano']==2024) & (merged['mes']==12) & ~(merged['match_mega1'].isnull()) & ~(merged['match_mega2'].isnull())].copy()

In [162]:
dec.drop_duplicates( subset=['bola1','bola2','bola3','bola4','bola5','bola6'], keep='first', inplace=True)

In [163]:
dec.shape

(8, 11)

In [164]:
dec.sort_values(by='Data')

,Data,parashat,bola1,bola2,bola3,bola4,bola5,bola6,match_mega1,match_mega2,match_mega3
51136,2024-12-20,וישב,26,15,16,22,19,49,MISPAR_KATAN,MISPAR_SIDURI,NaN
400440,2024-12-20,וישב,10,43,49,32,25,6,MISPAR_KATAN,MISPAR_KATAN,NaN
738840,2024-12-20,וישב,19,10,32,60,13,40,MISPAR_KATAN,MISPAR_SIDURI,NaN
761024,2024-12-20,וישב,52,56,4,24,46,55,MISPAR_KATAN,MISPAR_SIDURI,NaN
786968,2024-12-20,וישב,4,33,16,44,19,31,MISPAR_KATAN,MISPAR_SIDURI,NaN
851264,2024-12-20,וישב,50,20,46,23,14,39,MISPAR_KATAN,MISPAR_SIDURI,NaN
250418,2024-12-22,מקץ,5,23,16,4,49,1,ALBAM,ATBASH,NaN
115065,2024-12-29,ויגש,26,17,13,41,16,44,MISPAR_KATAN,MISPAR_SIDURI,NaN


In [1]:
11+20+31+34+35+43

174

In [3]:
2+4+12+33+41+47

139